In [26]:
import requests
from bs4 import BeautifulSoup
import re
import json
from config.config import ITEM_URL_BASE, BASE_URL, HOME_FILE, OUTPUT_FILE
from logger.logger import logger  # Import the configured logger

headers = {'User-Agent': 'Mozilla/5.0'}

def get_unique_item_ids(max_pages=3):
    """Fetch unique item IDs from paginated pages with proper logging."""
    item_ids = set()
    page = 1
    has_more_pages = True
    
    logger.info(f"Starting to scrape item IDs (max pages: {max_pages})")
    
    while has_more_pages:
        if max_pages and page > max_pages:
            logger.info(f"Reached max pages limit ({max_pages})")
            break
            
        url = f"{BASE_URL}?page={page}"
        logger.debug(f"Fetching page {page}: {url}")
        
        try:
            response = requests.get(url, headers=headers)
            
            if response.status_code != 200:
                logger.error(f"Failed to fetch page {page}. Status code: {response.status_code}")
                break

            soup = BeautifulSoup(response.text, "html.parser")
            
            # Find all item links and extract IDs from hrefs
            current_page_ids = set()
            for a in soup.select("a.item_link"):
                href = a.get("href")
                if href:
                    match = re.search(r'/items/(\d+)', href)
                    if match:
                        item_id = match.group(1)
                        current_page_ids.add(item_id)
            
            # Check if we found any new items on this page
            if not current_page_ids:
                logger.info(f"No items found on page {page}, stopping pagination")
                has_more_pages = False
            else:
                new_items = current_page_ids - item_ids
                if not new_items:
                    logger.info(f"No new items found on page {page}, stopping pagination")
                    has_more_pages = False
                else:
                    logger.info(f"Found {len(new_items)} new items on page {page}")
                    item_ids.update(current_page_ids)
                    page += 1
                    
        except requests.RequestException as e:
            logger.error(f"Network error fetching page {page}: {str(e)}", exc_info=True)
            break
        except Exception as e:
            logger.error(f"Unexpected error processing page {page}: {str(e)}", exc_info=True)
            break

    # Save raw HTML for debugging
    try:
        with open(HOME_FILE, mode="w", newline="", encoding="utf-8") as f:
            f.write(response.text)
        logger.debug(f"Saved last page HTML to {HOME_FILE}")
    except Exception as e:
        logger.error(f"Failed to save HTML to {HOME_FILE}: {str(e)}", exc_info=True)

    logger.info(f"Finished scraping. Found {len(item_ids)} unique items")
    return item_ids

# Run the function and print results
item_ids = get_unique_item_ids()
links = list(item_ids)
print(links)
print(len(links))

2025-05-09 02:32:50 - scraper - INFO - Starting to scrape item IDs (max pages: 3)
2025-05-09 02:32:51 - scraper - INFO - Found 28 new items on page 1
2025-05-09 02:32:51 - scraper - INFO - Found 28 new items on page 2
2025-05-09 02:32:52 - scraper - INFO - Found 28 new items on page 3
2025-05-09 02:32:52 - scraper - INFO - Reached max pages limit (3)
2025-05-09 02:32:52 - scraper - INFO - Finished scraping. Found 84 unique items


['5056827', '4959734', '5142259', '5165167', '5122759', '4802446', '5141958', '3852754', '5153399', '2716163', '5186675', '5169209', '4351437', '4939762', '5166148', '4912143', '5172433', '4617223', '4749417', '5094015', '5169083', '3533767', '4986500', '5165139', '4515580', '4881242', '5038500', '4624278', '5136085', '5097244', '5170034', '4908448', '5063184', '5111890', '4822793', '4959777', '5153278', '4999301', '4901100', '5153700', '5115965', '4884485', '4234996', '4965109', '4891042', '5060163', '3161506', '4280166', '5073997', '5032165', '5136086', '5166133', '5110014', '5181767', '5089925', '4983218', '3713070', '5064932', '5094044', '5176057', '4840131', '5058889', '5181362', '5158541', '5103258', '5163744', '5129119', '4997082', '5186450', '5174093', '5184531', '5147145', '5003954', '5095087', '5156675', '3309447', '5178713', '5184396', '5084859', '5140712', '5149053', '5161334', '5038503', '5126935']
84


In [27]:
def extract_item_data(link):
    """Extract detailed item data from individual product page with comprehensive logging"""
    url = ITEM_URL_BASE + link
    logger.debug(f"Starting to extract data from: {url}")
    
    try:
        response = requests.get(url, headers=headers)
        
        if response.status_code != 200:
            logger.error(f"Failed to fetch {url} - Status code: {response.status_code}")
            return None

        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract item ID
        item_id = link.split("/")[-1]
        logger.debug(f"Processing item ID: {item_id}")

        # Extract amount and currency using meta tags
        amount_tag = soup.find('meta', {'property': 'product:price:amount'})
        currency_tag = soup.find('meta', {'property': 'product:price:currency'})

        amount = amount_tag['content'] if amount_tag else None
        currency = currency_tag['content'] if currency_tag else None
        logger.debug(f"Extracted price: {amount} {currency}")

        # Extract property fields dynamically
        properties = {}
        prop_div = soup.find("div", class_="product-properties__column")
        if prop_div:
            for prop in prop_div.find_all("div", class_="product-properties__i"):
                try:
                    name = prop.find("label", class_="product-properties__i-name").get_text(strip=True)
                    value = prop.find("span", class_="product-properties__i-value").get_text(strip=True)
                    properties[name] = value
                    logger.debug(f"Found property: {name} = {value}")
                except Exception as e:
                    logger.warning(f"Failed to extract property: {str(e)}", exc_info=True)

        # Extract locations
        location_tags = soup.find_all('a', {'data-stat': 'product-locations'})
        locations = [tag.get_text(strip=True) for tag in location_tags] if location_tags else []
        logger.debug(f"Found locations: {locations}")

        # Construct result dictionary
        item_data = {
            'item_id': item_id,
            'amount': amount,
            'currency': currency,
            'location': locations
        }

        # Add all found properties dynamically
        item_data.update(properties)
        
        logger.info(f"Successfully processed item: {item_id}")
        return item_data

    except requests.RequestException as e:
        logger.error(f"Network error while processing {url}: {str(e)}", exc_info=True)
        return None
    except Exception as e:
        logger.error(f"Unexpected error processing {url}: {str(e)}", exc_info=True)
        return None


def scrape_all_items():
    """Main scraping function that processes all items with progress tracking"""
    if not links:
        logger.warning("No links to process - empty list provided")
        return

    results = []
    logger.info(f"Starting to process {len(links)} items")

    for idx, link in enumerate(links, 1):
        logger.info(f"[{idx}/{len(links)}] Processing: {link}")
        data = extract_item_data(link)
        if data:
            results.append(data)
            logger.debug(f"Added item {data.get('item_id')} to results")
        else:
            logger.warning(f"Failed to process item: {link}")

    # Save to JSON
    try:
        with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
            json.dump(results, f, indent=2, ensure_ascii=False)
        logger.info(f"Successfully saved {len(results)} items to {OUTPUT_FILE}")
        if len(results) != len(links):
            logger.warning(f"Processed {len(results)} out of {len(links)} items ({(len(results)/len(links))*100:.1f}% success rate)")
    except Exception as e:
        logger.error(f"Failed to save data to {OUTPUT_FILE}: {str(e)}", exc_info=True)


# Run the scraper
scrape_all_items()

2025-05-09 02:34:57 - scraper - INFO - Starting to process 84 items
2025-05-09 02:34:57 - scraper - INFO - [1/84] Processing: 5056827
2025-05-09 02:34:57 - scraper - INFO - Successfully processed item: 5056827
2025-05-09 02:34:57 - scraper - INFO - [2/84] Processing: 4959734
2025-05-09 02:34:58 - scraper - INFO - Successfully processed item: 4959734
2025-05-09 02:34:58 - scraper - INFO - [3/84] Processing: 5142259
2025-05-09 02:34:59 - scraper - INFO - Successfully processed item: 5142259
2025-05-09 02:34:59 - scraper - INFO - [4/84] Processing: 5165167
2025-05-09 02:35:00 - scraper - INFO - Successfully processed item: 5165167
2025-05-09 02:35:00 - scraper - INFO - [5/84] Processing: 5122759
2025-05-09 02:35:00 - scraper - INFO - Successfully processed item: 5122759
2025-05-09 02:35:00 - scraper - INFO - [6/84] Processing: 4802446
2025-05-09 02:35:01 - scraper - INFO - Successfully processed item: 4802446
2025-05-09 02:35:01 - scraper - INFO - [7/84] Processing: 5141958
2025-05-09 02: